In [1]:
import numpy as np
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
Y = iris.target

# Plus Proche Voisin

In [2]:
# 1
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import LeaveOneOut

def PPV(X,Y):
    loo = LeaveOneOut()
    res = []
    
    for train_index, test_index in loo.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        
        res.append((y_train[np.argmin(euclidean_distances(X_train,X_test).flatten())]))
        
    return res

In [3]:
# 2
def PPV(X,Y):
    loo = LeaveOneOut()
    res = []
    
    for train_index, test_index in loo.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        
        res.append((y_train[np.argmin(euclidean_distances(X_train,X_test).flatten())]))
        erreur = len([i for i in range(len(res)) if res[i] != Y[i]])/len(res)
        
    return res,erreur

In [4]:
# 3
y_pred,erreur = PPV(X,Y)
print(y_pred)
print("Erreur: ",erreur*100,"%")

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
Erreur:  4.0 %


In [5]:
# 4
from sklearn.neighbors import KNeighborsClassifier

def KNN(K):
    knn = KNeighborsClassifier(n_neighbors=K, metric='euclidean')
    loo = LeaveOneOut()
    res = []

    for train_index, test_index in loo.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]

        knn.fit(X_train, y_train)
        res.append(knn.predict(X_test)[0])

    erreur = len([res[i] for i in range(0, len(res)) if res[i] != Y[i]])/len(res)
    return erreur
    
    
print("Erreur:",KNN(1)*100,"%")
print("Remarque: Les résultats sont égaux pour K=1")

# pour autres valeurs de K
for i in range(2,10):
    print("Erreur pour k =",i,"est:",KNN(i)*100,"%")

Erreur: 4.0 %
Remarque: Les résultats sont égaux pour K=1
Erreur pour k = 2 est: 5.333333333333334 %
Erreur pour k = 3 est: 4.0 %
Erreur pour k = 4 est: 4.0 %
Erreur pour k = 5 est: 3.3333333333333335 %
Erreur pour k = 6 est: 4.0 %
Erreur pour k = 7 est: 3.3333333333333335 %
Erreur pour k = 8 est: 3.3333333333333335 %
Erreur pour k = 9 est: 3.3333333333333335 %


In [6]:
# 5
from collections import defaultdict
def PPV(X,Y,k=1):
    loo = LeaveOneOut()
    res=[]
    
    for train_index, test_index in loo.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        
        distances = euclidean_distances(X_train,X_test).flatten()
        indexs = np.argsort(distances)
        ppv = y_train[indexs][0:k]

        d = defaultdict(int)
        for e in ppv:
            d[e] += 1
            
        if list(d.values())[0]==1 or not all(x==list(d.values())[0] for x in list(d.values())):
            res.append(max(d,key=d.__getitem__))
            
        else: # cas: [1,1,2,2] || [1,2,1,1,2,2]
            moyenne_distances=defaultdict()
            for el in sorted(d):
                index_classes=[i for i, e in enumerate(ppv) if e == el]
                moyenne_distances[el] =sum(np.array(distances[indexs])[index_classes ])
            res.append(min(moyenne_distances, key=(lambda q: moyenne_distances[q])))
        
        erreur = len([res[i] for i in range(0, len(res)) if res[i] != Y[i]])/len(res)
    return res,erreur

for i in range(2,10):
    y_pred,erreur = PPV(X,Y,k=i)
    print("Erreur pour k =",i,"est:",erreur*100,"%")

Erreur pour k = 2 est: 4.0 %
Erreur pour k = 3 est: 4.0 %
Erreur pour k = 4 est: 4.0 %
Erreur pour k = 5 est: 3.3333333333333335 %
Erreur pour k = 6 est: 4.0 %
Erreur pour k = 7 est: 3.3333333333333335 %
Erreur pour k = 8 est: 3.3333333333333335 %
Erreur pour k = 9 est: 4.0 %


# Classifieur Bayesien Naïf

In [8]:
# 1
def CBN(X,Y):
    loo = LeaveOneOut()
    res = []
    for train_index, test_index in loo.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        
        # calcul de p_x_wk pour chaque variable 
        CBN = dict()
        
        # pour chaque variable
        for nb_features in range(X.shape[1]):
            p_variables = dict()
            
            # pour chaque classe
            for i in np.unique(Y):
                index_of_class_i = np.argwhere(Y==i).flatten()
                features_of_class_i = X[index_of_class_i,nb_features]
                p_variable_in_class = dict()
                for j in np.unique(features_of_class_i):
                    p_variable_in_class[j] = np.size(features_of_class_i[features_of_class_i==j])/np.size(features_of_class_i)
                p_variables["Class "+str(i)] = p_variable_in_class
            CBN["X"+str(nb_features)] = p_variables
        
        
        # les probabilités pour chaque classe existant
        p_class=dict()
        for i in np.unique(Y):
            p_class['Class '+str(i)]= np.size(Y[Y==i])/np.size(Y)

        # prediction
        for c in p_class:
            for v,w in zip(X_test.flatten(),[i for i in CBN]):
                try:
                    exist = CBN[w][c][v]
                except:
                    exist = 0
                p_class[c] *= exist

        res.append(max(p_class,key=p_class.__getitem__)[-1])
    return res
print(CBN(X,Y))

['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '2', '2', '2', '2', '2', '1', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '1', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2']


In [9]:
# 2
def CBN(X,Y):
    loo = LeaveOneOut()
    res = []
    for train_index, test_index in loo.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        
        # calcul de p_x_wk pour chaque variable 
        CBN = dict()
        
        # pour chaque variable
        for nb_features in range(X.shape[1]):
            p_variables = dict()
            
            # pour chaque classe
            for i in np.unique(Y):
                index_of_class_i = np.argwhere(Y==i).flatten()
                features_of_class_i = X[index_of_class_i,nb_features]
                p_variable_in_class = dict()
                for j in np.unique(features_of_class_i):
                    p_variable_in_class[j] = np.size(features_of_class_i[features_of_class_i==j])/np.size(features_of_class_i)
                p_variables["Class "+str(i)] = p_variable_in_class
            CBN["X"+str(nb_features)] = p_variables
        
        
        # les probabilités pour chaque classe existant
        p_class=dict()
        for i in np.unique(Y):
            p_class['Class '+str(i)]= np.size(Y[Y==i])/np.size(Y)

        # prediction
        for c in p_class:
            for v,w in zip(X_test.flatten(),[i for i in CBN]):
                try:
                    exist = CBN[w][c][v]
                except:
                    exist = 0
                p_class[c] *= exist

        res.append(max(p_class,key=p_class.__getitem__)[-1])
    erreur = len([i for i in range(0, len(res)) if int(res[i]) != Y[i]])/len(res)
    return res,erreur

y_pred,erreur = CBN(X,Y)
print("Erreur:",erreur*100,"%")

Erreur: 3.3333333333333335 %


In [10]:
# 3
from sklearn.naive_bayes import GaussianNB

loo = LeaveOneOut()
res=[]

for train_index, test_index in loo.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    clf = GaussianNB()
    clf.fit(X_train, y_train)
    res.append(clf.predict(X_test).flatten()[0])
erreur = len([i for i in range(0, len(res)) if int(res[i]) != Y[i]])/len(res)

print("Erreur=",erreur*100,"%")

Erreur= 4.666666666666667 %
